# Extract the adjacency matrix
This script is necessary to extract the adjacency matrix of the unstructured grid of a vtk file.

In [ ]:
import vtk
from vtk.util.numpy_support import vtk_to_numpy
from scipy import sparse

## Loading
The next cell loads an unstructured grid using vtk and stores it in the variable `assembly`

In [ ]:
reader = vtk.vtkXMLUnstructuredGridReader()
reader.SetFileName('../Variationsanalyse/data/sto_2//run__00001/m_1/vtk_Export/Assembly/Assembly_6.vtu')
reader.Update()
assembly = reader.GetOutput()

## Size estimation
Next, we need to know how many points this assembly contains to define a sparse matrix with that size.

In [ ]:
points_vtk = assembly.GetPoints().GetData()
points = vtk_to_numpy(points_vtk)
print(points.shape)

## Adjacency matrix creation
We can now create the adjacency matrix. For that, we iterate all cells to get their lines. For each line, we mark the connection of both points in the adjacency matrix. Please note, the adjacency matrix is symmetric, thus we could omit the last assignment to reduce the file size. But for the computation of the Fourier basis, a bidirectional (symmetric) adjacency matrix is necessary. We enforce the matrix to contain floats to allow an easy computation for the Fourier basis. Otherwise, we need to cast to a floating point type after we read the matrix back in the next script.

In [ ]:
A_sparse = sparse.lil_matrix((points.shape[0], points.shape[0]), dtype='f')
numCells = assembly.GetNumberOfCells()
for idx in range(numCells):
    cell = assembly.GetCell(idx)
    numLines = cell.GetNumberOfEdges()
    for idx in range(numLines):
        line = cell.GetEdge(idx)
        p1 = line.GetPointId(0)
        p2 = line.GetPointId(1)
        A_sparse[p1, p2] = 1
        A_sparse[p2, p1] = 1


## Storing

In [ ]:
sparse.save_npz('../Variationsanalyse/data/sto_2//adjacencyList.npz', A_sparse.tocsc())
